In [ ]:
#|default_exp callback.progresize

# Progressive Resizing
> A callback to progressively resize images during training

`ProgressiveResize` is inspired by MosaicML's [Progressive Resizing algorithm for Composer](https://docs.mosaicml.com/en/stable/method_cards/progressive_resizing.html) which in turn was inspired by [fastai](https://github.com/fastai/fastbook/blob/780b76bef3127ce5b64f8230fce60e915a7e0735/07_sizing_and_tta.ipynb). 

In [ ]:
#|export
from __future__ import annotations

from fastcore.basics import detuplify
from fastcore.transform import Pipeline, Transform

from fastai.callback.core import Callback
from fastai.vision.augment import AffineCoordTfm, RandomResizedCropGPU

from fastxtend.metrics import MetricX
from fastxtend.imports import *

In [ ]:
#|exporti
_resize_augs = (AffineCoordTfm, RandomResizedCropGPU)

In [ ]:
#|exporti
def _to_size(t:Tensor):
    "Convert Tensor to size compatible values"
    if sum(t.shape)==2: return tuple(t.tolist())
    else:               return tuple(t.item(),t.item())

In [ ]:
#|exporti
class PostProgResize(Callback):
    "Delete batch after resize to assist with PyTorch memory management"
    run_valid, order = True, 100 # Runs last

    def after_batch(self):
        if self.learn._progresize:
            del self.learn.xb
            del self.learn.yb
            del self.learn.pred
            self.learn._progresize = False

In [ ]:
#|export
class ProgressiveResize(Callback):
    run_valid, order = True, 5 # Needs to run before MixUp et al
    "Progressively increase the size of input images during training. Final image size is the valid image size."
    def __init__(self, 
        initial_size:Number|tuple[int,int]|None=0.5, 
        increase_by:int|tuple[int,int]=4, 
        start_at:float=0.5,
        finish_at:float=0.75,
        resize_mode:str='nearest',
        add_resize:bool=False,
        input_size:int|tuple[int,int]|None=None, # Final image size. Set if using a non-fastai Dataloaders.
        logger_callback='wandb' # Log report and samples/second to `logger_callback` using `Callback.name`
    ):
        store_attr()
        self._log_after_resize = getattr(self, f'_{self.logger_callback}_log_after_resize', noop)

    def before_fit(self):
        if hasattr(self.learn, 'lr_finder') and not hasattr(self, "gather_preds"): 
            self.run = False
            return

        self.learn._progresize = False
        if not hasattr(self.learn, 'post_prog_resize'): self.learn.add_cb(PostProgResize)
        self.remove_resize, self.null_resize, self.remove_cutmix = True, True, False
        self.has_logger = hasattr(self.learn, self.logger_callback)

        # Try to automatically determine the input size
        try:
            n_inp = self.dls.train.n_inp
            xb = self.dls.valid.one_batch()[:n_inp]
            for n in range(n_inp):
                x = detuplify(xb[n])
                if isinstance(x, TensorImageBase):
                    self.input_size = x.shape[-2:]
        finally:
            assert self.input_size is not None, "Could not determine input size. Set `input_size`."
            self.input_size = tensor(self.input_size)

        # Set the initial resize
        if isinstance(self.initial_size, float): 
            self.current_size = (tensor(self.initial_size) * self.input_size).int()
        elif isinstance(self.initial_size, tuple): 
            self.current_size = tensor(self.initial_size)

        # Double check that the step size works
        self.increase_by = tensor(self.increase_by)
        if ((self.input_size-self.current_size) % self.increase_by).sum() != 0:
            raise ValueError(f'Resize amount {self.input_size-self.current_size} not evenly divisible by `increase_by` {self.increase_by}')

        # Set when the progressive resizing step is applied
        n_steps = ((self.input_size-self.current_size) / self.increase_by).int()
        if len(n_steps.shape)==2: assert n_steps[0]==n_steps[1]
        pct = (self.finish_at - self.start_at) / (n_steps[0].item()-1)
        self.step_pcts = [self.start_at + pct*i for i in range(n_steps[0].item())]+[1.1]

        self._resize = []
        # If `add_resize`, add a seperate resize
        if self.add_resize:
            self._resize_pipe = Pipeline(AffineCoordTfm(size=_to_size(self.current_size), mode=self.resize_mode))
            self._resize.append(self._resize_pipe[0])
            self.remove_resize = True
        else:
            if hasattr(self.learn, 'cutmixupaugment'):
                # Modify the `CutMixUpAugment` augmentation pipeline 
                for i in range(len(self.learn.cutmixupaugment._orig_pipe)):
                    if isinstance(self.learn.cutmixupaugment._orig_pipe[i], _resize_augs):
                        self._resize.append(self.learn.cutmixupaugment._orig_pipe[i])
                        self.null_resize = self.null_resize and self.learn.cutmixupaugment._orig_pipe[i].size is None
                        self.remove_resize = False

                # If `CutMixUpAugment` has an Affine Transform for Augmentations then use it
                if len(self._resize) > 0:
                    # Check for pre-mixup augment pipeline and modify it
                    if self.learn.cutmixupaugment._docutmixaug:
                        for i in range(len(self.learn.cutmixupaugment._cutmixaugs_pipe)):
                            if isinstance(self.learn.cutmixupaugment._cutmixaugs_pipe[i], _resize_augs):
                                self._resize.append(self.learn.cutmixupaugment._cutmixaugs_pipe[i])
                                self.null_resize = self.null_resize and self.learn.cutmixupaugment._cutmixaugs_pipe[i].size is None
                                self.remove_resize = False    
                    else:
                        # There isn't one, then add it a pre-mixup augment pipeline for resizing
                        self.learn.cutmixupaugment._cutmixaugs_pipe = Pipeline(AffineCoordTfm(size=_to_size(self.current_size)))
                        self.learn.cutmixupaugment._docutmixaug = True
                        self._resize.append(self.learn.cutmixupaugment._cutmixaugs_pipe[0])
                        self.remove_cutmix, self.remove_resize = True, True

            else:
                # If no `CutMixUpAugment` check the dataloader pipeline for Affine Transforms
                for i in range(len(self.dls.train.after_batch.fs)):
                    if isinstance(self.dls.train.after_batch[i], _resize_augs):
                        self._resize.append(self.dls.train.after_batch[i])
                        self.null_resize = self.null_resize and self.dls.train.after_batch[i].size is None
                        self.remove_resize = False

            # If no there are no detected resizes add a resize transform pipeline
            if len(self._resize) == 0:
                self.add_resize = True
                self._resize_pipe = Pipeline(AffineCoordTfm(size=_to_size(self.current_size)))
                self._resize.append(self._resize_pipe[0])
                self.remove_resize = True

        # Set created or detected resize to the first size and store original interpolation
        self._orig_modes = []
        for resize in self._resize:
            resize.size = _to_size(self.current_size)
            self._orig_modes.append(resize.mode)
            resize.mode = self.resize_mode


    def before_batch(self):
        if self.add_resize:
            self.learn.xb = self._resize_pipe(self.xb)
            # self.learn.yb = self._resize_pipe(self.yb) TODO this wasn't working
        
    def after_batch(self):
        if self.pct_train >= self.step_pcts[0]:
            self.learn._progresize = True
            self.step_pcts = self.step_pcts[1:]
            self.current_size += self.increase_by
            for i, resize in enumerate(self._resize):
                if (self.current_size < self.input_size).all():
                    resize.size = _to_size(self.current_size)
                else:
                    # Reset everything after progressive resizing is done
                    if self.null_resize: 
                        resize.size = None
                    elif self.remove_resize:
                        if self.remove_cutmix:
                            self.learn.cutmixupaugment._cutmixaugs_pipe = Pipeline([])
                            self.learn.cutmixupaugment._docutmixaug = False
                        else:
                            self._resize_pipe = Pipeline([])
                            self.add_resize = False
                    else:
                        resize.size = _to_size(self.current_size)
                        resize.mode = self._orig_modes[i]
            if self.has_logger: self._log_after_resize()

In [ ]:
#|exports
try:
    import wandb

    @patch
    def _wandb_log_after_resize(self:ProgressiveResize):
        if len(self.current_size.shape)==2:
            size = {'progressive_resize_height': self.current_size[0],
                    'progressive_resize_width': self.current_size[1] }
        else:
            size = {'progressive_resize_size': self.current_size}
        wandb.log(size, self.learn.wandb._wandb_step+1)
except:
    pass

In [ ]:
#|hide
#|slow
from fastcore.basics import num_cpus

from fastai.data.external import URLs, untar_data
from fastai.data.block import DataBlock, CategoryBlock
from fastai.data.transforms import GrandparentSplitter, get_image_files, parent_label, Normalize
from fastai.learner import Learner
from fastai.vision.augment import Resize, aug_transforms
from fastai.vision.core import imagenet_stats
from fastai.vision.data import ImageBlock
from fastai.vision.models import resnet50
from fastxtend.callback.channelslast import *
from fastxtend.callback.cutmixup import CutMixUpAugment
from fastxtend.utils import *

In [ ]:
# @patch
# def _set_device(self:Learner, b):
#     model_device = next(self.model.parameters()).device
#     dls_device = getattr(self.dls, 'device', default_device())
#     if model_device == dls_device: return to_device(b, dls_device, non_blocking=True)
#     else: return to_device(b, model_device, non_blocking=True)

In [ ]:
#|hide
#|slow
class ProgressiveResizeTest(Callback):
    run_valid, order = False, ProgressiveResize.order+1
    
    def before_fit(self):
        self.progsize = self.learn.progressive_resize.current_size

    def before_batch(self):
        assert L(self.x.shape[-2:]) == L(self.progsize.tolist())
            
    def after_batch(self):
        self.progsize = self.learn.progressive_resize.current_size

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                    splitter=GrandparentSplitter(valid_name='val'),
                    get_items=get_image_files, get_y=parent_label,
                    item_tfms=Resize(256),
                    batch_tfms=[*aug_transforms(),Normalize.from_stats(*imagenet_stats)])
dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

learn = Learner(dls, resnet50(num_classes=dls.c)).to_channelslast()
learn.fit_one_cycle(20, 1e-3)

In [ ]:
#|slow
#|cuda
free_gpu_memory(learn, dls)

In [ ]:
#|slow
#|cuda
imagenette = untar_data(URLs.IMAGENETTE_320)

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                    splitter=GrandparentSplitter(valid_name='val'),
                    get_items=get_image_files, get_y=parent_label,
                    item_tfms=Resize(256),
                    batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
dls =  dblock.dataloaders(imagenette, bs=64, num_workers=num_cpus(), pin_memory=True)

cbs = [ProgressiveResize(0.5, increase_by=8, start_at=0.1, finish_at=0.85)]
learn = Learner(dls, resnet50(num_classes=dls.c), cbs=cbs).to_channelslast()
learn.fit_one_cycle(20, 1e-3)